In [ ]:
import os
import torch
import pickle

from dataset import YouCookII
from dataset import YouCookIICollate
from torch.utils.data import DataLoader

In [ ]:
def pickle_data(data, pickles_dir, fname):
    """
    Pickle data into bytestreams
    data: data to be pickled
    pickles_dir: directory path to pickled data
    fname: name of pickled file
    """
    if not os.path.isdir(pickles_dir):
        os.mkdir(pickles_dir)
    pickle_out = open(os.path.join(pickles_dir, fname + '.pickle'), 'wb')
    pickle.dump(data, pickle_out)
    pickle_out.close()
    
def depickle_data(pickles_dir, fname):
    pickle_path = os.path.join(pickles_dir, fname + '.pickle')
    pickle_in = open(pickle_path, 'rb')
    data = pickle.load(pickle_in)
    return data

In [ ]:
MAX_DETECTIONS = 20
FRAMES = 5

COORDINATES = 4
EMBEDDING_SIZE = 2048

root = "/h/sagar/ece496-capstone/datasets/ycii"
collate = YouCookIICollate(MAX_DETECTIONS=MAX_DETECTIONS)
num_actions = sorted(list(map(int, os.listdir(root))))

In [ ]:
video_id_list = []
bboxes_tensor = None
features_tensor = None
steps_list = []
entity_list = []
entity_count_list = []
step_length_list = []

for num_action in num_actions:
    print("Processing videos of size {}".format(num_action))
    
    dataset = YouCookII(num_action, "/h/sagar/ece496-capstone/datasets/ycii")
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=collate)
    
    for data in dataloader:
        video_id, bboxes, features, actions, _, entities, entity_count, max_step_length = data
        
        video_id = video_id[0]
        bboxes = bboxes[0]
        features = features[0]
        actions = actions[0]
        entities = entities[0]
        entity_count = entity_count[0]
                
        for i in range(num_action):
            video_id_list.append(video_id)
            step_length_list.append(len(actions[i].split()))
            
        if bboxes_tensor is None:
            bboxes_tensor = bboxes
        else:
            bboxes_tensor = torch.cat((bboxes_tensor, bboxes))

        if features_tensor is None:
            features_tensor = features
        else:
            features_tensor = torch.cat((features_tensor, features))
            
        steps_list = steps_list + actions
        entity_list = entity_list + entities
        entity_count_list = entity_count_list + entity_count

print(len(video_id_list), len(steps_list), len(entity_list), len(entity_count_list), len(step_length_list))
print(bboxes_tensor.shape[0], features_tensor.shape[0])

In [ ]:
chunk = 128

video_id_chunks = [video_id_list[i:(i+chunk)] for i in range(0, len(video_id_list), chunk)][:-1]
steps_chunks = [steps_list[i:(i+chunk)] for i in range(0, len(steps_list), chunk)][:-1]
entity_chunks = [entity_list[i:(i+chunk)] for i in range(0, len(entity_list), chunk)][:-1]
entity_count_chunks = [entity_count_list[i:(i+chunk)] for i in range(0, len(entity_count_list), chunk)][:-1]
step_length_chunks = [step_length_list[i:(i+chunk)] for i in range(0, len(step_length_list), chunk)][:-1]

bboxes_chunks = bboxes_tensor.split(MAX_DETECTIONS * FRAMES * chunk)[:-1]
features_chunks = features_tensor.split(MAX_DETECTIONS * FRAMES * chunk)[:-1]

print(len(video_id_chunks), len(steps_chunks), len(entity_chunks), len(entity_count_chunks), len(step_length_chunks))
print(len(bboxes_chunks), len(features_chunks))

In [ ]:
print("Total number of steps: {}".format(chunk * len(steps_chunks)))

In [ ]:
root = "/h/sagar/ece496-capstone/datasets/ycii_{}/{}".format(chunk, chunk)
os.makedirs(root, exist_ok=True)

In [ ]:
for idx, data in enumerate(zip(video_id_chunks, steps_chunks, entity_chunks, entity_count_chunks, step_length_chunks, bboxes_chunks, features_chunks)):
    print("Processing pseudo-video {}".format(idx))
    
    video_ids, actions, entities, entity_counts, step_lengths, bboxes, features = data
    
    #     bboxes = bboxes.reshape(-1, MAX_DETECTIONS, COORDINATES).unsqueeze(1)
    #     features = features.reshape(-1, MAX_DETECTIONS, EMBEDDING_SIZE).unsqueeze(1)
    #     candidates = [(data[0], data[1]) for data in zip(bboxes, features)]
    
    steps = '. '.join(actions) + '.'
    max_step_length = max(step_lengths)
    
    bboxes = bboxes.clone()
    features = features.clone()
    
    dir = os.path.join(root, str(idx).zfill(5), 'pickles')
    os.makedirs(dir, exist_ok=True)
    
    pickle_data(video_ids, dir, 'vid_id')
    pickle_data(bboxes, dir, 'bboxes')
    pickle_data(features, dir, 'features')
    pickle_data(actions, dir, 'actions_list')
    pickle_data(steps, dir, 'steps')
    pickle_data(entities, dir, 'entities')
    pickle_data(entity_counts, dir, 'entity_count')
    pickle_data(max_step_length, dir, 'max_step_length')

In [ ]:
dataset = YouCookII(4, "/h/sagar/ece496-capstone/datasets/ycii_4")
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=collate)

In [ ]:
for idx, data in enumerate(dataloader):
    print("Fetching pseudo-video {}".format(idx))
    video_id, bboxes, features, actions, steps, entities, entity_count, max_step_length = data